# SPAMMS Animations

Once your SPAMMS models have been calculated, it can be very imformative to visualize how the line profiles vary with the orientation of your system.  In the case of semi-detached or contact binaries these phase differences can have quite a large effect on the observed line profiles.  In this notebook, we demonstrate how to create an animation to visualize this.

In [ ]:
# Lets import the necessary packages...
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import PolyCollection
from matplotlib import cm, colors
import matplotlib.animation as animation
%matplotlib notebook
plt.rcParams['figure.dpi'] = 100
import spamms as sp

In any given SPAMMS run, you could be calculating several different models so we need to specify which model we wish to visualize.  Lets start here:

In [ ]:
output_folder = 'Outputs/Demo_semidetached/'
model_number = 'Model_0002'
He_abundance = '0.1'
CNO_abundance = '7.5'

Now that we have this, we can grab the necessary information from the input file and calculate the PHOEBE models for the system corresponding to the information from that model

In [ ]:
# grab the input file and use the builtin SPAMMS functions to read and interpret the input file
input_file = output_folder + 'input.txt'

fit_param_values, abund_param_values, line_list, io_dict = sp.read_input_file(input_file)
times, obs_specs = sp.get_obs_spec_and_times(io_dict)

# Creates the dictionary for each combination of parameters as defined in the input file
run_dictionaries = sp.create_runs_and_ids(fit_param_values)

Based on the model number (which corresponds to the dictionary number) we can grab the model information that we need (orbital and stellar parameters) and run the PHOEBE model


In [ ]:
# Grab the model info and run the PHOEBE Model
run_dictionary = run_dictionaries[int(model_number.split('_')[-1])]
s = sp.run_b_phoebe_model(times, abund_param_values, io_dict, run_dictionary)

Now we can gather the data that we need for the animation. To do so, lets start by looping through the PHOEBE models and the SPAMMS models to grab the relevant info that we need for each frame.

In [ ]:
# Lets start with the PHOEBE models.

# We need the geometrical info (the polygons that we will plot to visualize the mesh)
polygons_all = []

# We need the face color for the mesh triangles.  Lets use temperature here
teffs_all = []
rvs_all = []
loggs_all = []


# We'll loop through each time step that the PHOEBE model has and grab the 
# relevant info and populate the arrays above
for i in range(len(s['times@dataset@lc'].value)):
    print('%i/%i'%(i, len(s['times@dataset@lc'].value) - 1), end="\r")
          
    phcb = s['%09.6f'%s['times@dataset@lc'].value[i]]

    verts = np.concatenate([phcb['mesh@primary@mesh01@uvw_elements'].get_value(), 
                            phcb['mesh@secondary@mesh01@uvw_elements'].get_value()])
    teffs = np.concatenate([phcb['mesh@primary@mesh01@teffs'].get_value(), 
                            phcb['mesh@secondary@mesh01@teffs'].get_value()])
    loggs = np.concatenate([phcb['mesh@primary@mesh01@loggs'].get_value(), 
                            phcb['mesh@secondary@mesh01@loggs'].get_value()])
    rvs = np.concatenate([phcb['mesh@primary@rvs'].get_value(), 
                          phcb['mesh@secondary@rvs'].get_value()])

    xs = verts[:, :, 0]
    ys = verts[:, :, 1]
    zs = verts[:, :, 2]

    polygons = np.concatenate((xs[:,:,np.newaxis], ys[:,:,np.newaxis]), axis=2)


    viss = np.concatenate([phcb['visibilities@primary'].get_value(), 
                           phcb['visibilities@secondary'].get_value()])
    inds = (viss > 0)

    z_values = np.concatenate([phcb['ws@primary'].get_value(), 
                               phcb['ws@secondary'].get_value()])
    zinds = np.argsort(z_values[inds])
    
    polygons_all.append(polygons[inds][zinds])
    rvs_all.append(rvs[inds][zinds])



To deal with the SPAMMS models, we'll use the same function that we used in the previous notebook:

In [ ]:
def SPAMMS_lines_stitch(w1, f1, w2, f2):
    # To be used with SPAMMS spectral lines.  This function stitches together spectral lines by multiplying
    # normalized fluxes.  If there is no overlap region then the lines are just appended
    final_wave = []
    final_flux = []
    
    if w1[-1] < w2[0]:
        #if no overlap then
        final_wave.extend(w1)
        final_wave.extend(w2)
        final_flux.extend(f1)
        final_flux.extend(f2)
    else:
        # Define a wavelength array that covers both lines and interpolate both to the same wavelength array
        # Then multiply the fluxes together and return final wavelength and flux arrays
        
        final_wave = np.arange(w1[0], w2[-1]+0.001, 0.01)
        
        f1_interp = np.interp(final_wave, w1, f1, left=1.0, right=1.0)
        f2_interp = np.interp(final_wave, w2, f2, left=1.0, right=1.0)
        
        final_flux = f1_interp * f2_interp
        
        
    return final_wave, final_flux


def compile_SPAMMS_spectra(output_folder, model_number, He_abundance, CNO_abundance):
    # Load in the relevant information
    input_file = output_folder + 'input.txt'

    fit_param_values, abund_param_values, line_list, io_dict = sp.read_input_file(input_file)
    times, obs_specs = sp.get_obs_spec_and_times(io_dict)

    run_dictionaries = sp.create_runs_and_ids(fit_param_values)
    run_dictionary = run_dictionaries[int(model_number.split('_')[-1])]
    
    # define the path to the individual lines
    path_to_lines = output_folder + model_number + '/He%s_CNO%s/'%(He_abundance, CNO_abundance)
    
    # define our wavelength and flux lists where we will append the spectrum from each timestep
    wavelengths = []
    fluxes = []
    
    #loop through each time step
    for i in range(len(times)):

        line_wavelengths = []
        line_fluxes = []

        # loop through and grab each line
        for line in line_list:
            w,f = np.loadtxt(path_to_lines + 'hjd' + str(round(times[i], 13)).ljust(13, '0') + '_' + line + '.txt').T
            line_wavelengths.append(w)
            line_fluxes.append(f)
            
        # order the lines by wavelength to make stitching them together easier
        inds = np.array(np.argsort([i[0] for i in line_wavelengths]))

        # lets combine the lines to make a spectrum
        wavelength = line_wavelengths[inds[0]]
        flux = line_fluxes[inds[0]]
        for j in range(1, len(inds)):
            wavelength, flux = SPAMMS_lines_stitch(wavelength, flux, line_wavelengths[inds[j]], line_fluxes[inds[j]])

        wavelengths.append(wavelength)
        fluxes.append(flux)
    
    return wavelengths, fluxes


In [ ]:
wavelengths, fluxes = compile_SPAMMS_spectra(output_folder, model_number, He_abundance, CNO_abundance)

Now we can start actually making the plots and the animation

In [ ]:
# define our axes
fig, axs = plt.subplots(2,1, figsize = (6,9))

# calculate our minimum and maximum temperature to scale our colorbar
min_teff = min([min(i) for i in teffs_all])
max_teff = max([max(i) for i in teffs_all])
norm = colors.Normalize(vmin=min_teff, vmax=max_teff)
fig.colorbar(cm.ScalarMappable(norm=norm, cmap='inferno'), ax=axs[0])

# create a collection of polygons using the first frame
pc = PolyCollection(polygons_all[0],
                    edgecolors=cm.inferno(norm(teffs_all[0])),
                    facecolors=cm.inferno(norm(teffs_all[0])))

# add the collection to our axes and make some adjustments to the bounds
axs[0].add_collection(pc)
axs[0].set_aspect('equal')
axs[0].axis('scaled')

axs[0].set_xlim((-24,24))
axs[0].set_ylim((-16,16))
# axs[0].axis('off')

# add in the spectra to the other axes
line, = axs[1].plot(wavelengths[0], fluxes[0], c='k')
# axs[1].set_xlim((4425,4585))
axs[1].set_ylim((0.6, 1.05))
axs[1].set_xlabel('Wavelength ($\AA$)')
axs[1].set_ylabel('Normalized Flux')

# define our animation function
def animate(i):
    pc.set_verts(polygons_all[i])
    pc.set_edgecolors(cm.inferno(norm(teffs_all[i])))
    pc.set_facecolors(cm.inferno(norm(teffs_all[i])))
    line.set_data(wavelengths[i], fluxes[i])

    
def init():
    pc.set_verts([])
    line.set_data([],[])
    return pc, line,

# call the animation function, but lets leave off the last frame as its the same as the first
ani = animation.FuncAnimation(fig, animate, np.arange(0, len(times) -1), init_func=init,
                              interval=100, blit=True)
plt.show()